In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from glove import Glove
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import tqdm
import gc
import os
import sys
sys.path.append("../../../code/utils/")
sys.path.append("../../../code/analysis/")
sys.path.append('../../../code/pipeline/')
sys.path.append('../../../code')
import data_jointer as dj
import eval_utils as eu
import data_utils as du
import perf_utils as pu
import config

In [2]:
user_jointer = dj.PandasMatrixJointer(on="uid")
df_train = du.load_raw_data("train")
df_test = du.load_raw_data("test2")
df_all = pd.concat([df_train, df_test], ignore_index=True)

train_size = df_train.shape[0]
test_size = df_test.shape[0]


def run_embedding(feat_name, emb_dim=10, learning_rate=0.01, epochs=300, version_no=1):
    # =========
    # load data
    # =========
    word_to_index, cooc = du.load_preliminary_user_feature_coocurrence(feat_name)
    row_uids, (_, matrix_bin) = du.load_user_cnt(feat_name)
    num_users = matrix_bin.shape[0]
    print("Number of Users in Binary Matrix: {}".format(num_users))

    # ===============
    # train embedding
    # ===============
    with pu.profiler("fitting GloVe embedding"):
        glove = Glove(no_components=emb_dim, 
                      learning_rate=learning_rate, 
                      random_state=2018)  # fix random_state for reproducibility
        glove.fit(cooc.tocoo().astype(np.float64), 
                  epochs=epochs,
                  verbose=False)
        col_names = ["{}_embedding_{}".format(feat_name, i) for i in range(emb_dim)]

    # ====================
    # save embedding model
    # ====================
    glove_folder = "../../../model/glove"
    glove_folder = os.path.join(glove_folder, "[featureName='{}']".format(feat_name))
    glove_file = "v{}.glove_model".format(version_no)
    glove_path = os.path.join(glove_folder, glove_file)
    os.makedirs(glove_folder, exist_ok=True)
    du.save_pickle(glove, glove_path)

    # ==========
    # do pooling
    # ==========
    # should use multiprocessing
    with pu.profiler("preparing avg/max/min pooling of embedding"):
        matrix_avg = np.zeros((num_users, emb_dim))
        matrix_min = np.zeros((num_users, emb_dim))

        for i, row in tqdm.tqdm(enumerate(matrix_bin), total=num_users, disable=True):
            row_pool = glove.word_vectors[row.indices]
            matrix_avg[i] = row_pool.mean(axis=0)
            matrix_min[i] = row_pool.min(axis=0)

        matrix_avg = matrix_avg.astype(np.float32)
        matrix_min = matrix_min.astype(np.float32)

    # ===========
    # join matrix
    # ===========
    with pu.profiler("joining avg pooling matrix"):
        X_avg = user_jointer.join(df_all, matrix_avg, row_names=row_uids)
        X_avg = X_avg.astype(np.float32)
        del matrix_avg
        gc.collect()

    with pu.profiler("joining min pooling matrix"):
        X_min = user_jointer.join(df_all, matrix_min, row_names=row_uids)
        X_min = X_min.astype(np.float32)
        del matrix_min
        gc.collect()
    
    # =========
    # save data
    # =========
    emb_folder = "../../../data/embedding"
    emb_folder = os.path.join(emb_folder, "[featureName='{}']".format(feat_name))
    os.makedirs(emb_folder, exist_ok=True)

    with pu.profiler("saving data"):
        # avg pooling
        emb_file = "train.avg_v{}.pkl".format(version_no)
        emb_path = os.path.join(emb_folder, emb_file)
        du.save_pickle((col_names, X_avg[:train_size]), emb_path)  # save train data
        
        emb_file = "test2.avg_v{}.pkl".format(version_no)
        emb_path = os.path.join(emb_folder, emb_file)
        du.save_pickle((col_names, X_avg[train_size:]), emb_path)  # save test data

        # min pooling
        emb_file = "train.min_v{}.pkl".format(version_no)
        emb_path = os.path.join(emb_folder, emb_file)
        du.save_pickle((col_names, X_min[:train_size]), emb_path)  # save train data
        
        emb_file = "test2.min_v{}.pkl".format(version_no)
        emb_path = os.path.join(emb_folder, emb_file)
        du.save_pickle((col_names, X_min[train_size:]), emb_path)  # save test data

        del X_avg
        del X_min
        gc.collect()

In [3]:
user_multi_feat_names = config.USER_MULTI_FEAT_NAMES

In [4]:
run_embedding("marriageStatus", 
              emb_dim=13, 
              learning_rate=0.001, 
              epochs=1000, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[15:22:58] Finish fitting GloVe embedding. △M: +136.0KB. △T: 0.1 seconds.
[15:36:47] Finish preparing avg/max/min pooling of embedding. △M: +1.11GB. △T: 13.8 minutes.
[15:37:18] Finish joining avg pooling matrix. △M: +2.8MB. △T: 31.0 seconds.
[15:37:48] Finish joining min pooling matrix. △M: -28.93MB. △T: 29.9 seconds.
[15:37:49] Finish saving data. △M: -1.07GB. △T: 0.8 seconds.


In [5]:
run_embedding("interest1", 
              emb_dim=20, 
              learning_rate=0.0002, 
              epochs=5000, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[15:38:13] Finish fitting GloVe embedding. △M: +12.0KB. △T: 22.2 seconds.
[15:52:33] Finish preparing avg/max/min pooling of embedding. △M: +1.7GB. △T: 14.3 minutes.
[15:53:08] Finish joining avg pooling matrix. △M: -6.86MB. △T: 35.0 seconds.
[15:53:41] Finish joining min pooling matrix. △M: -26.86MB. △T: 33.8 seconds.
[15:53:43] Finish saving data. △M: -1.65GB. △T: 1.2 seconds.


In [6]:
run_embedding("interest2", 
              emb_dim=14, 
              learning_rate=0.0002, 
              epochs=5000, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[15:53:51] Finish fitting GloVe embedding. △M: +4.0KB. △T: 8.2 seconds.
[16:06:57] Finish preparing avg/max/min pooling of embedding. △M: +1.19GB. △T: 13.1 minutes.
[16:07:25] Finish joining avg pooling matrix. △M: -18.98MB. △T: 27.8 seconds.
[16:07:53] Finish joining min pooling matrix. △M: -18.97MB. △T: 28.0 seconds.
[16:07:54] Finish saving data. △M: -1.15GB. △T: 1.0 seconds.


In [7]:
run_embedding("interest3", 
              emb_dim=10, 
              learning_rate=0.01, 
              epochs=300, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[16:07:55] Finish fitting GloVe embedding. △M: +0B. △T: 0.0 seconds.
[16:20:42] Finish preparing avg/max/min pooling of embedding. △M: +871.35MB. △T: 12.8 minutes.
[16:21:12] Finish joining avg pooling matrix. △M: -1.49MB. △T: 29.4 seconds.
[16:21:41] Finish joining min pooling matrix. △M: -13.55MB. △T: 28.8 seconds.
[16:21:41] Finish saving data. △M: -844.17MB. △T: 0.7 seconds.


In [8]:
run_embedding("interest4", 
              emb_dim=10, 
              learning_rate=0.01, 
              epochs=300, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[16:21:42] Finish fitting GloVe embedding. △M: +0B. △T: 0.0 seconds.
[16:37:15] Finish preparing avg/max/min pooling of embedding. △M: +871.28MB. △T: 15.6 minutes.
[16:38:00] Finish joining avg pooling matrix. △M: +30.89MB. △T: 44.8 seconds.
[16:38:45] Finish joining min pooling matrix. △M: -13.55MB. △T: 45.6 seconds.
[16:38:46] Finish saving data. △M: -844.17MB. △T: 0.8 seconds.


In [9]:
run_embedding("interest5", 
              emb_dim=10, 
              learning_rate=0.01, 
              epochs=300, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[16:38:51] Finish fitting GloVe embedding. △M: +4.0KB. △T: 2.3 seconds.
[17:03:13] Finish preparing avg/max/min pooling of embedding. △M: +871.29MB. △T: 24.4 minutes.
[17:04:02] Finish joining avg pooling matrix. △M: +22.32MB. △T: 49.6 seconds.
[17:04:56] Finish joining min pooling matrix. △M: -13.55MB. △T: 53.6 seconds.
[17:04:57] Finish saving data. △M: -844.17MB. △T: 1.0 seconds.


In [10]:
run_embedding("ct", 
              emb_dim=5, 
              learning_rate=0.001, 
              epochs=1000, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[17:04:59] Finish fitting GloVe embedding. △M: +0B. △T: 0.2 seconds.
[17:28:14] Finish preparing avg/max/min pooling of embedding. △M: +435.64MB. △T: 23.2 minutes.
[17:29:07] Finish joining avg pooling matrix. △M: +14.09MB. △T: 52.8 seconds.
[17:29:53] Finish joining min pooling matrix. △M: -6.78MB. △T: 46.2 seconds.
[17:29:53] Finish saving data. △M: -422.09MB. △T: 0.4 seconds.


In [ ]:
run_embedding("os", 
              emb_dim=2, 
              learning_rate=0.001, 
              epochs=1000, 
              version_no=1)

Number of Users in Binary Matrix: 11420039
[17:29:55] Finish fitting GloVe embedding. △M: +0B. △T: 1.1 seconds.
[17:50:39] Finish preparing avg/max/min pooling of embedding. △M: +174.26MB. △T: 20.7 minutes.
[17:51:17] Finish joining avg pooling matrix. △M: +44.41MB. △T: 38.6 seconds.
[17:52:01] Finish joining min pooling matrix. △M: -2.71MB. △T: 43.4 seconds.
[17:52:01] Finish saving data. △M: -168.84MB. △T: 0.2 seconds.


In [ ]:
run_embedding("kw1", 
              emb_dim=50, 
              learning_rate=0.01, 
              epochs=100, 
              version_no=1)

Number of Users in Binary Matrix: 11420039


In [ ]:
run_embedding("kw2", 
              emb_dim=50, 
              learning_rate=0.01, 
              epochs=300, 
              version_no=1)

In [ ]:
run_embedding("kw3", 
              emb_dim=50, 
              learning_rate=0.001, 
              epochs=5000, 
              version_no=1)

In [ ]:
run_embedding("topic1", 
              emb_dim=50, 
              learning_rate=0.01, 
              epochs=300, 
              version_no=1)

In [ ]:
run_embedding("topic2", 
              emb_dim=50, 
              learning_rate=0.01, 
              epochs=300, 
              version_no=1)

In [ ]:
run_embedding("topic3", 
              emb_dim=50, 
              learning_rate=0.0005, 
              epochs=6000, 
              version_no=1)

In [ ]:
run_embedding("appIdInstall", 
              emb_dim=30, 
              learning_rate=0.1, 
              epochs=1, 
              version_no=1)

In [ ]:
run_embedding("appIdAction", 
              emb_dim=50, 
              learning_rate=0.01, 
              epochs=300, 
              version_no=1)